##### 1.1 regis_model_fuel_df: all fuel types (for 2.1)
##### 1.2 regis_model_fuel_e_df: electric (for analysis)
##### 2.1 regis_model_state_df: all fuel types (use 1.1 to check join)
##### 2.2 regis_model_state_e_df: electric (for analysis)


In [4]:
# start 2019

### 1. model_and_fuel_type

#### 1.1 regis_model_fuel_df

In [5]:
# import
import pandas as pd

In [6]:
# read
regis_model_fuel_df = pd.read_excel('model_19.xlsx', sheet_name = 'model_and_fuel_type')
regis_model_fuel_df['Hersteller'] = regis_model_fuel_df['Hersteller'].fillna(method = 'ffill')

# drop the first column
regis_model_fuel_df = regis_model_fuel_df.drop(['Unnamed: 0'], axis = 1)

# drop the rows with ZUSAMMEN
regis_model_fuel_df = regis_model_fuel_df.drop(regis_model_fuel_df[regis_model_fuel_df['Hersteller'].str.contains('ZUSAMMEN')].index)

# rename
regis_model_fuel_df.rename(columns={'Hersteller': 'manufacturer',
                                            'Handelsname': 'model',
                                            'Typ-Schl.-Nr.': 'tsn',
                                            'kW': 'power_kw',
                                            'Kraftstoffart': 'fuel_type',
                                            'Allrad': 'drive_type',
                                            'Aufbauart': 'body_type',
                                            'Insgesamt': 'new_registration_per_model',
                                            'Wohnmobile': 'motorhomes',
                                            'private\nHalter': 'private_owners',
                                            'Halter\nbis 29 Jahre': 'owners_under_29_years',
                                            'Halter\nab 60 Jahre': 'owners_over_60_years',
                                            'weibliche\nHalter': 'female_owners'}, inplace=True)

In [7]:
regis_model_fuel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3205 entries, 0 to 3263
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   manufacturer                3205 non-null   object 
 1   model                       3204 non-null   object 
 2   tsn                         3146 non-null   object 
 3   power_kw                    3146 non-null   float64
 4   fuel_type                   3146 non-null   object 
 5   drive_type                  1120 non-null   object 
 6   body_type                   166 non-null    object 
 7   new_registration_per_model  3205 non-null   int64  
 8   motorhomes                  3205 non-null   object 
 9   private_owners              3205 non-null   object 
 10  owners_under_29_years       3205 non-null   object 
 11  owners_over_60_years        3205 non-null   object 
 12  female_owners               3205 non-null   object 
dtypes: float64(1), int64(1), object(1

In [8]:
# check NaN
list = ['model', 'new_registration_per_model']
rows_with_nan = regis_model_fuel_df.loc[regis_model_fuel_df[list].isna().any(axis=1)]
rows_with_nan

,manufacturer,model,tsn,power_kw,fuel_type,drive_type,body_type,new_registration_per_model,motorhomes,private_owners,owners_under_29_years,owners_over_60_years,female_owners
3263,SONSTIGE HERSTELLER,NaN,NaN,NaN,NaN,NaN,NaN,2203,385,733,27,234,216


In [9]:
# Getting rid of leading or trailing whitespaces
regis_model_fuel_df = regis_model_fuel_df.applymap(lambda x: x.strip() if isinstance(x, str) else x) 

# Variable with all columns to change
cols_to_convert_fuel = ['new_registration_per_model', 'motorhomes', 'private_owners',
            'owners_under_29_years', 'owners_over_60_years', 'female_owners']

# Replace - with 0 and , with .
for col_fuel in cols_to_convert_fuel:
    regis_model_fuel_df[col_fuel] = regis_model_fuel_df[col_fuel].replace('-', '0').replace(',', '.').astype(int)

# Checking for duplicates
duplicates = regis_model_fuel_df.duplicated()
print('Number of duplicate entries: ', duplicates.sum())
print(duplicates)

Number of duplicate entries:  0
0       False
1       False
2       False
3       False
4       False
        ...  
3258    False
3259    False
3260    False
3261    False
3263    False
Length: 3205, dtype: bool


In [10]:
# drop duplicates
regis_model_fuel_df = regis_model_fuel_df.drop_duplicates()

In [11]:
# add new columns
regis_model_fuel_df['business_owners'] = regis_model_fuel_df['new_registration_per_model'] - regis_model_fuel_df['private_owners']
regis_model_fuel_df['owners_between_29_and_60_years'] = regis_model_fuel_df['new_registration_per_model'] - regis_model_fuel_df['owners_under_29_years'] - regis_model_fuel_df['owners_over_60_years']
regis_model_fuel_df['male_owners'] = regis_model_fuel_df['new_registration_per_model'] - regis_model_fuel_df['female_owners']
regis_model_fuel_df["year"] = 2019

In [12]:
# drop columns
regis_model_fuel_df = regis_model_fuel_df.drop(['power_kw', 'drive_type', 'body_type', 'motorhomes'], axis = 1)

In [13]:
regis_model_fuel_df = regis_model_fuel_df.reset_index(drop=True)

## All fuel types if needed
# regis_model_fuel_df.to_csv('regis_model_fuel_df_2019.csv', index=False)

#### 1.2 regis_model_fuel_e_df 


In [14]:
# concentrate on only electric cars
regis_model_fuel_e_df = regis_model_fuel_df[regis_model_fuel_df['fuel_type'] == 'E']

In [15]:
# final
regis_model_fuel_e_df  = regis_model_fuel_e_df.reset_index(drop=True)
regis_model_fuel_e_df.to_csv('regis_model_fuel_e_df_2019.csv', index=False)

### 2.model_by_state

#### 2.1 regis_model_state_df

In [16]:
# read
regis_model_state_df = pd.read_excel('model_19.xlsx', sheet_name = 'model_by_state')
regis_model_state_df['Hersteller'] = regis_model_state_df['Hersteller'].fillna(method = 'ffill')

# drop the first column
regis_model_state_df = regis_model_state_df.drop(['Unnamed: 0'], axis = 1)

# drop the rows with ZUSAMMEN 
regis_model_state_df = regis_model_state_df.drop(regis_model_state_df[regis_model_state_df['Hersteller'].str.contains('ZUSAMMEN')].index)

# rename
regis_model_state_df.rename(columns={'Hersteller': 'manufacturer',
                                            'Handelsname': 'model',
                                            'Typ-Schl.-Nr.': 'tsn',
                                            'Baden-\nWürttemberg' : 'Baden-Württemberg',
                                            'Bayern': 'Bayern',
                                            'Berlin': 'Berlin',
                                            'Branden-\nburg': 'Brandenburg',
                                            'Bremen': 'Bremen',
                                            'Hamburg': 'Hamburg',
                                            'Hessen': 'Hessen',
                                            'Mecklenburg-\nVorpommern': 'Mecklenburg-Vorpommern',
                                            'Nieder-\nsachsen': 'Niedersachsen',
                                            'Nordrhein-\nWestfalen': 'Nordrhein-Westfalen',
                                            'Rheinland-\nPfalz': 'Rheinland-Pfalz',
                                            'Saarland': 'Saarland',
                                            'Sachsen': 'Sachsen',
                                            'Sachsen-\nAnhalt': 'Sachsen-Anhalt',
                                            'Schleswig-\nHolstein': 'Schleswig-Holstein',
                                            'Thüringen': 'Thüringen',
                                            'Sonstige': 'special',
                                            'Deutschland': 'new_registration_per_model'}, inplace=True)

In [17]:
regis_model_state_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3205 entries, 0 to 3263
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   manufacturer                3205 non-null   object
 1   model                       3204 non-null   object
 2   tsn                         3146 non-null   object
 3   Baden-Württemberg           3205 non-null   object
 4   Bayern                      3205 non-null   object
 5   Berlin                      3205 non-null   object
 6   Brandenburg                 3205 non-null   object
 7   Bremen                      3205 non-null   object
 8   Hamburg                     3205 non-null   object
 9   Hessen                      3205 non-null   object
 10  Mecklenburg-Vorpommern      3205 non-null   object
 11  Niedersachsen               3205 non-null   object
 12  Nordrhein-Westfalen         3205 non-null   object
 13  Rheinland-Pfalz             3205 non-null   obje

In [18]:
# Nan
rows_with_nan = regis_model_state_df.loc[regis_model_state_df['new_registration_per_model'].isna()]
rows_with_nan

,manufacturer,model,tsn,Baden-Württemberg,Bayern,Berlin,Brandenburg,Bremen,Hamburg,Hessen,...,Niedersachsen,Nordrhein-Westfalen,Rheinland-Pfalz,Saarland,Sachsen,Sachsen-Anhalt,Schleswig-Holstein,Thüringen,special,new_registration_per_model


In [19]:
regis_model_state_df = regis_model_state_df.dropna(subset=['new_registration_per_model'])

In [20]:
regis_model_state_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3205 entries, 0 to 3263
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   manufacturer                3205 non-null   object
 1   model                       3204 non-null   object
 2   tsn                         3146 non-null   object
 3   Baden-Württemberg           3205 non-null   object
 4   Bayern                      3205 non-null   object
 5   Berlin                      3205 non-null   object
 6   Brandenburg                 3205 non-null   object
 7   Bremen                      3205 non-null   object
 8   Hamburg                     3205 non-null   object
 9   Hessen                      3205 non-null   object
 10  Mecklenburg-Vorpommern      3205 non-null   object
 11  Niedersachsen               3205 non-null   object
 12  Nordrhein-Westfalen         3205 non-null   object
 13  Rheinland-Pfalz             3205 non-null   obje

In [21]:
# Variable with all columns to change
cols_to_convert_state = ['Baden-Württemberg', 'Bayern',
            'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen',
            'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen',
            'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt',
            'Schleswig-Holstein', 'Thüringen', 'special']
# Getting rid of leading or trailing whitespaces
regis_model_state_df = regis_model_state_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
 # Replace - with 0
for col_state in cols_to_convert_state:
    regis_model_state_df[col_state] = regis_model_state_df[col_state].replace('-', '0').replace(',', '.').astype(float).fillna(0).astype(int)
# Checking for duplicates
duplicates_state = regis_model_state_df.duplicated()
print('Number of duplicate entries: ', duplicates.sum())
print(duplicates_state)
        

Number of duplicate entries:  0
0       False
1       False
2       False
3       False
4       False
        ...  
3258    False
3259    False
3260    False
3261    False
3263    False
Length: 3205, dtype: bool


In [22]:
# drop duplicates
regis_model_state_df = regis_model_state_df.drop_duplicates()

In [23]:
#  want to add fuel type but need to check (ues data from 1.1)
regis_model_state_df = regis_model_state_df.reset_index(drop=True)
regis_model_state_df["year"] = 2019
regis_model_state_df['check'] = regis_model_fuel_df['new_registration_per_model']
regis_model_state_df[regis_model_state_df.new_registration_per_model != regis_model_state_df.check] 

,manufacturer,model,tsn,Baden-Württemberg,Bayern,Berlin,Brandenburg,Bremen,Hamburg,Hessen,...,Rheinland-Pfalz,Saarland,Sachsen,Sachsen-Anhalt,Schleswig-Holstein,Thüringen,special,new_registration_per_model,year,check


In [24]:
# if there is no results for last cell, then continue
# add fuel type
regis_model_state_df = regis_model_state_df.drop(['check'], axis = 1)
regis_model_state_df['fuel_type'] = regis_model_fuel_df['fuel_type']

In [25]:
regis_model_state_df['model'] = regis_model_fuel_df['model']
regis_model_state_df['tsn'] = regis_model_fuel_df['tsn']

In [26]:
# put all states into one column
id_cols = pd.concat([regis_model_state_df.iloc[:, :3], regis_model_state_df.iloc[:, 20:]], axis=1)
regis_model_state_df = pd.melt(regis_model_state_df, id_vars=id_cols, value_vars=regis_model_state_df.columns[3:20], var_name='federal_state', value_name='new_registration')
regis_model_state_df = regis_model_state_df.drop(['new_registration_per_model'], axis = 1)

In [27]:
# final
regis_model_state_df = regis_model_state_df.reset_index(drop=True)

## All fuel types if needed
# regis_model_state_df.to_csv('regis_model_state_df_2019.csv', index=False)

#### 2.2 regis_model_state_e_df

In [28]:
regis_model_state_e_df = regis_model_state_df[regis_model_state_df.fuel_type == 'E']

In [29]:
# final
regis_model_state_e_df = regis_model_state_e_df.reset_index(drop=True)
regis_model_state_e_df.to_csv('regis_model_state_e_df_2019.csv', index=False)

## Inserting DataFrames into PostgreSQL Server

In [30]:
# Import get_engine function from sql_functions.py and set it to a variable called engine
from sql_functions import get_engine
engine = get_engine()
# Import psycopg2
import psycopg2

In [33]:
# IMPORTANT: Set the schema to capstone schema name and set the table_name variable
schema = 'cp_analytics_g2' # example 'hh_analytics_22_1
table_name = 'new_registration_2019_fuel'

In [34]:
# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        regis_model_fuel_e_df.to_sql(table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # your class schema
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The new_registration_2019_fuel table was imported successfully.


In [35]:
# Define second table name variable
schema_2 = 'cp_analytics_g2' # example 'hh_analytics_22_1
table_name_2 = 'new_registration_2019_state'

In [36]:
# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        regis_model_state_e_df.to_sql(table_name_2, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema_2, # your class schema
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name_2} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The new_registration_2019_state table was imported successfully.
